# Assigment 02

Now that we already have the data from previous assgiment, let import the files and start to work with them.

In [1]:
#Import the librarys
import pandas as pd

In [2]:
data = pd.read_pickle('./data/raw/10-K_Data.pik')

In [3]:
data.head()

,index,company_name,form_type,cik,date,file,corpus
0,275433,TRAVELZOO INC,10-K,1133311,2017-03-15,edgar/data/1133311/0001133311-17-000010.txt,b'<SEC-DOCUMENT>0001133311-17-000010.txt : 201...
1,2116,ACCO BRANDS Corp,10-K,712034,2017-02-27,edgar/data/712034/0000712034-17-000012.txt,b'<SEC-DOCUMENT>0000712034-17-000012.txt : 201...
2,127781,GrowGeneration Corp.,10-K,1604868,2017-03-31,edgar/data/1604868/0001213900-17-003102.txt,b'<SEC-DOCUMENT>0001213900-17-003102.txt : 201...
3,16371,Advanced Biomedical Technologies Inc.,10-K,1385799,2017-02-14,edgar/data/1385799/0001387131-17-000831.txt,b'<SEC-DOCUMENT>0001387131-17-000831.txt : 201...
4,230264,"Primerica, Inc.",10-K,1475922,2017-02-27,edgar/data/1475922/0001564590-17-002594.txt,b'<SEC-DOCUMENT>0001564590-17-002594.txt : 201...


## Removing the html tags

The files we have imported have html tags and it is very dificult to read, lets tale a look

In [5]:
sample = data['corpus'][0]
print(sample[300:700])

b'15\nDATE AS OF CHANGE:\t\t20170315\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tTRAVELZOO INC\n\t\tCENTRAL INDEX KEY:\t\t\t0001133311\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tSERVICES-COMPUTER INTEGRATED SYSTEMS DESIGN [7373]\n\t\tIRS NUMBER:\t\t\t\t364415727\n\t\tSTATE OF INCORPORATION:\t\t\tDE\n\t\tFISCAL YEAR END:\t\t\t1231\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t10-K\n\t\tSEC ACT:\t\t1934 Act\n\t\tSEC FILE NUMBER:\t000-50171\n\t\tFILM N'


In [9]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [10]:
def text_process( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string and the output is a single string
    
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review,"html5lib").get_text() 
    
    # 2. Remove non-letters
    review_text = re.sub('\s+', ' ',review_text)    
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)

    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

In [11]:
# Check to make sure its working
sample_clean = data['corpus'].head(1).apply(text_process)
list(sample_clean)[0][300:700]

'orporation de fiscal year end filing values form type k sec act act sec file number film number business address street madison avenue street th floor city new york state ny zip business phone mail address street madison avenue street th floor city new york state ny zip k tzoo x k htm k document united statessecurities exchange commissionwashington c form k mark one xannual report pursuant section'

In [12]:
# Get the number of documents based on the dataframe column size
num_documents = data['corpus'].count()
corpus = data['corpus']
clean_corpus = []

# Loop over each document; create an index i that goes from 0 to the 
#length of documents 
for i in range( 0, num_documents ):
    # Call our function for each one, and add the result to the 
    #list of clena documents
    clean_corpus.append( text_process( corpus[i] ) )
    
    # Printing out the the progress. 
    try:
        count += 1
        end = time.time()
    except:
        import sys 
        import time
        start = time.time()
        end = time.time()
        count = 1
        
        #Need to change this variavel to match with the range of the for loop
        num_documents = data['corpus'].count() 
        
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * count/float(num_documents)) \
                     + "% ... Documents processed: " + str(count) \
                     + " time running: " + str(int((end-start)/60)) + " minutes.") 
    sys.stdout.flush()

Progress: 100.0% ... Documents processed: 100 time running: 105 minutes.

In [15]:
data['corpus'] = clean_corpus

Lets save the file already clened, so next time we can start from here.

In [16]:
data.to_pickle('./data/10k-clean.pik')

# Importing the new data file

In [1]:
import pandas as pd
data = pd.read_pickle('./data/10k-clean.pik')

In [2]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

## Creating Stemmed and Lemmatized Corpus

In [3]:
#Creating a new collum for the corpus stemmed and lemmatized
corpus_stemed = []
corpus_lemmatized = []

for corpus in data['corpus']:
    corpus_stemed.append(" ".join([stemmer.stem(word) \
                                   for word in corpus.split()]))
    corpus_lemmatized.append(" ".join([WordNetLemmatizer.lemmatize(word,word) \
                                   for word in corpus.split()]))
    
    # Printing out the the progress. 
    try:
        count += 1
        end = time.time()
    except:
        import sys 
        import time
        start = time.time()
        end = time.time()
        count = 1
        
        #Need to change this variavel to match with the range of the for loop
        num_documents = data['corpus'].count() 
        
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * count/float(num_documents)) \
                     + "% ... Documents processed: " + str(count) \
                     + " time running: " + str(int((end-start)/60)) + " minutes.") 
    sys.stdout.flush()
    
    
data['stemmed'] = corpus_stemed
data['lemmatized'] = corpus_lemmatized

Progress: 100.0% ... Documents processed: 100 time running: 33 minutes.

## Creating Word Count and Unique Count

In [4]:
#Function to calculate word count
def word_count(data):
    return len(data.split())

#Function to calculate unique words
def unique_count(data):
    return len(set(data.split()))

In [5]:
#Creating a new collum for the corpus word count and unique words
data['word count'] = data['corpus'].apply(word_count)

#Creating a new collumn for uniques words
data['unique words'] = data['corpus'].apply(unique_count)
data['unique stemmed'] = data['stemmed'].apply(unique_count)
data['unique lemmatizes'] = data['lemmatized'].apply(unique_count)

Now Lets Count Negatives Words

In [6]:
url_neg = 'https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/08a269765a6b185d5f3dd522c876043ba9628715/data/opinion-lexicon-English/negative-words.txt'
neg_words = pd.read_fwf(url_neg,encoding='latin-1',skiprows=34,header=None, 
                       names=['negatives'])

In [7]:
neg_stemmed = []
neg_lemmatized = []

for word in neg_words['negatives']:
    neg_stemmed.append(stemmer.stem(word))
    neg_lemmatized.append(WordNetLemmatizer.lemmatize(word,word))

neg_words['stemm'] = neg_stemmed
neg_words['lemmatizes'] = neg_lemmatized

In [8]:
neg_words.head()

,negatives,stemm,lemmatizes
0,2-faced,2-face,2-faced
1,2-faces,2-face,2-faces
2,abnormal,abnorm,abnormal
3,abolish,abolish,abolish
4,abominable,abomin,abominable


In [9]:
neg_word = set(list(neg_words['negatives']))
neg_stemmed = set(list(neg_words['stemm']))
neg_lemmatizes = set(list(neg_words['lemmatizes']))

In [10]:
def negative_count (data, negative=neg_word):
   return len([word for word in data.split() if word in negative])

In [11]:
data['Negative words'] = data['corpus'].apply(negative_count,neg_word)
data['Negative Stemmed'] = data['stemmed'].apply(negative_count,negative=neg_stemmed)
data['Negative lemmatized'] = data['lemmatized'].apply(negative_count,negative=neg_lemmatizes)

Lets save our database for future use

In [12]:
data.to_pickle('./data/10k-v2.pik')

We will export some columns to csv file, first we will select the columns, than export

In [13]:
data[['company_name','word count','unique words','unique stemmed', \
      'unique lemmatizes', 'Negative words', 'Negative Stemmed', \
      'Negative lemmatized']].head()


,company_name,word count,unique words,unique stemmed,unique lemmatizes,Negative words,Negative Stemmed,Negative lemmatized
0,TRAVELZOO INC,327222,12039,10608,11529,1733,4309,3179
1,ACCO BRANDS Corp,841296,28456,25464,27566,5243,12004,10317
2,GrowGeneration Corp.,175052,11434,10226,11011,1178,3157,2550
3,Advanced Biomedical Technologies Inc.,82050,8008,6845,7601,889,2321,1635
4,"Primerica, Inc.",132391,10091,7831,9352,3017,6600,4301


In [14]:
#exporting
data[['company_name','word count','unique words','unique stemmed', \
      'unique lemmatizes', 'Negative words', 'Negative Stemmed', \
      'Negative lemmatized']].to_csv('./data/word count.csv')